In [382]:
import numpy as np

class sudoku:
    @staticmethod
    def torow(block,order):
        return (block//3)*3+order//3

    @staticmethod
    def tocolumn(block,order):
        return (block%3)*3+order%3

    @staticmethod
    def toblock(row,column):
        return (row//3)*3+column//3

    __board_flags=np.zeros(9*9,dtype=np.ubyte).reshape(9,9)
    __board_options=np.ones(9*9*9,dtype=bool).reshape(9,9,9)
    
    __number_of_solutions=0
    __limit_of_solutions=10

    def print_board(self):
       # print(self.__board_flags)
        for i in range(9):
            for j in range(9):
                flag=self.__board_flags[i][j]
                if flag==0:
                    for k in range(9):
                        option=self.__board_options[i][j][k]
                        if option==True:
                            print(k+1,end="")
                        else:
                            print("-",end="")
                else:
                    for k in range(9):
                        if k+1==flag:
                            print(flag,end="")
                        else:
                            print("=",end="")
                print("  ",end="")
                if j==2 or j==5:
                    print("   ",end="")
            print()
            if i==2 or i==5:
                print()
        print()
        print()
        
    def __fix_board_flag(self,i,j,flag):
        self.__board_options[i][j].fill(False)
        self.__board_options[i][j][flag-1]=True
        self.__board_flags[i][j]=flag
        for k in range(9):
            if self.__board_flags[k][j]==0:
                self.__board_options[k][j][flag-1]=False
                if self.__board_options[k][j].sum()==0:
                    return False
            block=self.toblock(i,j)
            if self.__board_flags[i][k]==0:
                self.__board_options[i][k][flag-1]=False
                if self.__board_options[i][k].sum()==0:
                    return False
            row=self.torow(block,k)
            column=self.tocolumn(block,k)
            if self.__board_flags[row][column]==0:
                self.__board_options[row][column][flag-1]=False
                if self.__board_options[row][column].sum()==0:
                    return False
        return True

    def __fix_board_allflags(self):
        while True:
            action=False
            for i in range(9):
                for j in range(9):
                    flag=self.__board_flags[i][j]
                    if flag==0 and self.__board_options[i][j].sum()==1:
                        newflag=np.where(self.__board_options[i][j])[0][0]+1
                        # print("(",i,",",j,")=",newflag," ",self.__board_options[i][j])
                        if self.__fix_board_flag(i,j,newflag)==False:
                            return False
                        action=True

            for i in range(9):
                for d in range(9):
                    row_options=self.__board_options[i,0:9,d]
                    if row_options.sum()==1:
                        column=np.where(row_options)[0][0]
                        if self.__board_flags[i,column]==0:
                            #print(i+1,column+1,d+1)
                            #self.print_board()
                            if self.__fix_board_flag(i,column,d+1)==False:
                                return False
                            action=True
            
            for j in range(9):
                 for d in range(9):
                    column_options=self.__board_options[0:9,j,d]
                    if column_options.sum()==1:
                        row=np.where(column_options)[0][0]
                        if self.__board_flags[row,j]==0:
                            #print(row+1,j+1,d+1)
                            #self.print_board()
                            if self.__fix_board_flag(row,j,d+1)==False:
                                return False
                            action=True
                            
            for k in range(9):
                 for d in range(9):
                    row_top=self.torow(k,0)
                    column_left=self.tocolumn(k,0)
                    block_options=self.__board_options[row_top:row_top+3,column_left:column_left+3,d]
                    if block_options.sum()==1:
                        row_relative=np.where(block_options)[0][0]
                        column_relative=np.where(block_options)[1][0]
                        row=row_top+row_relative
                        column=column_left+column_relative
                        if self.__board_flags[row,column]==0:
                            #print("block:",k+1,",d:",d+1,sep="")
                            #print("cornor:(",row_top+1,",",column_left+1,")",sep="")
                            #print(np.where(block_options))
                            #print("position:(",row_relative+1,",",column_relative+1,"),d:",sep="")
                            #self.print_board()
                            if self.__fix_board_flag(row,column,d+1)==False:
                                return False
                            action=True
                             
            if action==False:
                break
        return True 

    def __initialize_board_flags(self,initial_board=None):
        if type(initial_board)==type([]):
            initial_board_flags=initial_board
        elif type(initial_board)==type("0"):
            initial_board_flags=[]
            for line in initial_board.split('\n'):
                if len(initial_board_flags)>=9: break
                initial_board_flags.append([])
                for ch in line:
                    if ch.isdigit():
                        if len(initial_board_flags[-1])==9:
                            initial_board_flags.append([])
                        initial_board_flags[-1].append(int(ch))
                    if ch==",":
                        while len(initial_board_flags[-1])<9:
                            initial_board_flags[-1].append(0)
            while len(initial_board_flags[-1])<9:
                initial_board_flags[-1].append(0)
        else:
            initial_board_flags=[]
            while len(initial_board_flags)<9:
                line=input("please input row %d: "%(len(initial_board_flags)+1))
                initial_board_flags.append([])
                for ch in line:
                    if ch.isdigit():
                        if len(initial_board_flags[-1])==9:
                            initial_board_flags.append([])
                        initial_board_flags[-1].append(int(ch))
                    if ch==",":
                        while len(initial_board_flags[-1])<9:
                            initial_board_flags[-1].append(0)
            while len(initial_board_flags[-1])<9:
                initial_board_flags[-1].append(0)
        #print(initial_board_flags)
        self.__board_flags=np.array(initial_board_flags,dtype=np.ubyte)
        #print(self.__board_flags)

    
    def __initialize_board_options(self):
        self.__board_options=np.ones(9*9*9,dtype=bool).reshape(9,9,9)
        for i in range(9):
            for j in range(9):
                flag=self.__board_flags[i][j]
                if flag>0:
                    # print("board({row},{column})={flag}".format(row=i,column=j,flag=self.__board_flags[i][j]))
                    if self.__fix_board_flag(i,j,flag)==False:
                        return False
        if self.__fix_board_allflags()==False:
            return False
        print("Initialize:")
        self.print_board()
        return True

    def __search_all_solutions(self):
        if self.__number_of_solutions>=self.__limit_of_solutions:
            return;
        while True:
            action=False
            row=column=0
            for i in range(9):
                for j in range(9):
                    if self.__board_flags[i][j]==0:
                        action=True
                        row=i
                        column=j
                        break
            if action==True:
                options=self.__board_options[row][column]
                options_number=options.sum()
                options_digits=np.where(options)[0]
                temp_board_flags=self.__board_flags.copy()
                temp_board_options=self.__board_options.copy()
                for digit in options_digits:
                    self.__board_flags=temp_board_flags.copy()
                    self.__board_options=temp_board_options.copy()
                    #print("try(",row+1,",",column+1,")=",digit+1,", ",sep="",end="")
                    if self.__fix_board_flag(row,column,digit+1)==False:
                        #print("Fail!")
                        continue
                    if self.__fix_board_allflags()==False:
                        #print("Fail!")
                        continue
                    #self.self.print_board()
                    self.__search_all_solutions()
                return
            else:
                print("Success!")
                self.print_board()
                self.__number_of_solutions+=1               
                return 
        
    def __init__(self,initial_board=None):
        self.initialize(initial_board)
    
    def initialize(self,initial_board=None):
        self.__initialize_board_flags(initial_board)
        self.__initialize_board_options()    
        
    def solve(self,limit=10):
        self.__limit_of_solutions=limit
        self.__number_of_solutions=0
        self.__search_all_solutions()
        print("number_of_solutions: ",self.__number_of_solutions)
        

In [383]:
board1=[[6,0,0,3,1,0,0,4,0],[0,0,1,0,0,0,0,7,5],[7,4,2,0,5,0,0,0,0],\
        [0,7,4,5,0,0,2,0,9],[0,0,9,7,0,4,5,3,1],[0,0,0,0,0,1,0,6,0],\
        [4,8,6,2,7,0,1,0,0],[3,0,5,0,0,6,7,0,4],[0,0,7,4,3,9,0,5,8]]
board2="794386020638000749215947836,073600098,900803467,08607901,80976005,30009867,067030984"
board3="000500260305000004090001000020040600000012005900830000000700100800000047057009000"
board4="600000017 400001600 001000000 000010970 310800064 062004001 108097046 000000100 200100705"
board6="00007619,972158463,061040507,59000267,70800523,006700045,600007004,00003075,00700031"
board7="0204007,006709,080002006,230000095,0,090000017,010205048,5020043,0083"
board8="10000709,030020008,0096005,0053009,010080002,600004,30000001,041000007,0070003" # very hard
board9="000000039,000001005,0030508,008090006,070002,1004,00908005,0200006,4007" # very hard
board0="070000004,60000009,0080031,0000153,000302,00586,0015002,090000006,40000007" # extremely hard
boarda="0007008,00004003,000009001,6005,01003004,005001007,5002006,03008009,007000002" # very hard
boardb="0900817,0000008,800007012,207,000506,000000903,580300004,001,00480006" # hard
boardc="100000089,000009002,00000045,0076,03004,900002005,00407,50000801,0603" # hard

sdk=sudoku(board4)
sdk.solve()


Initialize:
=====6===  -23-5--89  --3-5---9     -2345---9  -2345--8-  -23-5--89     -2345--8-  1========  ======7==  
===4=====  -23-5-789  --3-5-7-9     -23-5-7-9  -23-5-78-  1========     =====6===  -23-5--89  -23----89  
----5-789  -23-5-789  1========     -234567-9  -2345678-  -23-56-89     -2345--8-  -23-5--89  -23----89  

----5--8-  ---45--8-  ---45----     -23-56---  1========  -23-56---     ========9  ======7==  -23----8-  
==3======  1========  ----5-7-9     =======8=  -2--5-7--  -2--5---9     -2--5----  =====6===  ===4=====  
----5-789  =====6===  =2=======     --3-5-7-9  --3-5-7--  ===4=====     --3-5--8-  --3-5--8-  1========  

1========  --3-5----  =======8=     -23-5----  ========9  ======7==     -23------  ===4=====  =====6===  
----5-7-9  --345-7-9  --34567-9     -23456---  -23456-8-  -23-56-8-     1========  -23----89  -23----89  
=2=======  --34----9  --34-6--9     1========  --34-6-8-  --3--6-8-     ======7==  --3----89  ====5====  


Success!
=====6===  =======8= 